In [1]:
import ollama

### Saving the conversation in a List and using it for future references

In [7]:
def ollama_chat(messages):
    response = ollama.chat(
        model='llama3.1',
        messages=messages)
    return response['message']['content'], response['message']

In [2]:
# Having an initial conversation and storing it in a list named messages

messages = []

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    
    messages.append({'role': 'user', 'content': user_input})
    response, message = ollama_chat(messages)
    messages.append({'role': 'assistant', 'content': response})
    
    print("Bot:", response)
    print('-----'*10)

You: Hey, I am from South India. Explain its climate and top 5 special places.
Bot: Namaste!

South India, which comprises the states of Tamil Nadu, Kerala, Karnataka, Andhra Pradesh, and Telangana, has a unique and diverse climate due to its geographical location.

**Climate:**

The climate in South India varies from region to region, but generally, it is characterized by:

1. **Tropical wet climate**: The coastal regions experience high temperatures and high humidity throughout the year.
2. **Monsoon season**: The southwest monsoon brings heavy rainfall to the western parts of Tamil Nadu, Karnataka, and Kerala from June to September.
3. **Hot and dry climate**: The interiors of Andhra Pradesh, Telangana, and southern Tamil Nadu have a hot and dry climate during summer (March to May).
4. **Winter months**: From October to February, the temperatures are relatively mild in most parts of South India.

**Top 5 Special Places:**

Here are five must-visit places in South India:

1. **Meenak

In [4]:
# Loading the conversation developed earlier and feeding it to the model.

def start_chat_with_model(messages):
    introduction_message = {
        'role': 'system',
        'content': "The following 'messages' list contains our previous conversation. We will continue our chat from here."
    }
    messages.insert(0, introduction_message)  

    response, _ = ollama_chat(messages)
    return response

start_chat_with_model(messages)

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    
    messages.append({'role': 'user', 'content': user_input})
    response, _ = ollama_chat(messages)
    messages.append({'role': 'assistant', 'content': response})
    
    print("Bot:", response)
    print('-----' * 10,'\n')

You: explain about the second temple you suggested for me in 5lines.
Bot: You're referring to the **Backwaters of Alleppey (Alappuzha), Kerala**!

Here's a brief summary:

* The backwaters are a serene and picturesque network of waterways, canals, and lakes.
* Take a relaxing houseboat ride through the tranquil countryside, spotting local villages and wildlife.
* Explore the lush greenery, coconut groves, and paddy fields surrounding the waters.
* Enjoy delicious Keralan cuisine and refreshing coconut water on board.
* Experience the essence of rural Kerala's simple yet charming way of life.
-------------------------------------------------- 

You: quit


### Storing the conversation in SQLite mapping it using the session id and using it for future references.

In [1]:
import sqlite3
import ollama

In [2]:
conn = sqlite3.connect('chat_history_.db')
cursor = conn.cursor()

# Create table for storing chat messages
cursor.execute('''
CREATE TABLE IF NOT EXISTS chat_messages_ (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    role TEXT,
    content TEXT
)
''')

conn.commit()

In [3]:
def save_message(session_id, role, content):
    cursor.execute('''
    INSERT INTO chat_messages_ (session_id, role, content)
    VALUES (?, ?, ?)
    ''', (session_id, role, content))
    conn.commit()
    
def load_messages(session_id):
    cursor.execute('''
    SELECT role, content FROM chat_messages_
    WHERE session_id = ?
    ORDER BY id ASC
    ''', (session_id,))
    return [{'role': row[0], 'content': row[1]} for row in cursor.fetchall()]

In [10]:
# Having a conversation and storing it in the session_id

session_id = 'session_aug30'

# Load previous messages
messages_sql = load_messages(session_id)

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    
    messages_sql.append({'role': 'user', 'content': user_input})
    save_message(session_id, 'user', user_input)
    
    response, message = ollama_chat(messages_sql)
    messages_sql.append({'role': 'assistant', 'content': response})
    save_message(session_id, 'assistant', response)
    
    print("Bot:", response)
    print('-----'*10)

conn.close()

You: Tell top 5 places to visit in south india
Bot: A fellow South Indian! I'd be happy to recommend the top 5 special places to visit in South India. Here they are:

**1. Meenakshi Amman Temple, Madurai (Tamil Nadu)**
This ancient Hindu temple is a must-visit for its stunning architecture, intricate carvings, and vibrant atmosphere. The temple complex has over 2,000 pillars, 4 concentric quadrangles, and 12 gopurams (towers).

**2. Hampi, Karnataka**
A UNESCO World Heritage Site, Hampi is an ancient city that was once the capital of the Vijayanagara Empire. Explore the ruins of temples, palaces, markets, and other structures that showcase a rich history.

**3. Backwaters of Alleppey (Alappuzha), Kerala**
Experience the tranquility of South India's backwaters on a houseboat or by walking through the lush green countryside. The serene waterways are home to a variety of flora and fauna.

**4. Mahabalipuram, Tamil Nadu**
This coastal town is famous for its ancient rock-cut temples, includ

In [15]:
# Loading the previous conversation and continuing the chat from the initial response it provided.

conn = sqlite3.connect('chat_history_.db')
cursor = conn.cursor()

messages_sql = load_messages(session_id)

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    
    messages_sql.append({'role': 'user', 'content': user_input})
    save_message(session_id, 'user', user_input)
    
    response, message = ollama_chat(messages_sql)
    messages_sql.append({'role': 'assistant', 'content': response})
    save_message(session_id, 'assistant', response)
    
    print("Bot:", response)
    print('-----'*10)

conn.close()

You: why the 4th place is so special?
Bot: The 4th place, Mahabalipuram, is considered special for several reasons:

* **Ancient rock-cut temples**: The Shore Temple (circa 8th century AD) is one of the oldest and most well-preserved examples of rock-cut architecture in India. It's a testament to the skill and craftsmanship of ancient Indian artisans.
* **Historical significance**: Mahabalipuram was an important port city during the Pallava Dynasty, which ruled southern India from the 3rd to the 9th century AD. The town is a reminder of the rich cultural heritage of this period.
* **Unique sculptures**: The Pancha Pandava Rathas (five chariot-like structures) are another highlight of Mahabalipuram. These intricate sculptures depict scenes from Hindu mythology and are considered some of the finest examples of ancient Indian art.

These factors combined make Mahabalipuram a special place, attracting visitors interested in history, architecture, and culture.
------------------------------

**_Happy Learning!!!_**